# Skills examples 

This notebook is intended to highlight the different capabilities and skills of our models. This is not an exhaustive list, and the examples are meant to be simple so it's easy to understand what's going on. Note we are mostly using Lite across the examples, because of this.

We will not focus on the details around how to invoke the models and what is supported, for that please refer to the Getting Started notebook.

---

## Setup

Please make sure you followed the steps outlines in the Quick Start first. Namely, that you have the model enabled on your account and region

**Note**: _Below Examples are Text Understanding use cases and can be used with Micro, Lite, Pro, Premier models. You can try other models changing the `DEFAULT_MODEL_ID`_

## Prerequisites

Run the cells in this section to install the required packages.  

_IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts._

In [ ]:
%pip install -q --force-reinstall \
    "botocore>=1.40.26" \
    "awscli>=1.29.57" \
    "boto3>=1.40.26"

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
from botocore.client import Config
import json
from pprint import pprint

LITE_MODEL_ID = "us.amazon.nova-2-lite-v1:0"

DEFAULT_MODEL_ID = LITE_MODEL_ID

client = boto3.client("bedrock-runtime", 
                      region_name="us-east-1",
                      config=Config(read_timeout=10000)
                    )

def invoke(prompt, model_id=DEFAULT_MODEL_ID, maxtokens=1000,reasoning="high"):
    messages = [
        {"role": "user", "content": [{"text": prompt}]},
    ]

    if reasoning == "high":
        inference_config = {
            "reasoningConfig": {
                "type": "enabled",
                "maxReasoningEffort": reasoning
            }
        } 
    else:
        inference_config = {
            "maxTokens": maxtokens, 
            "topP": 0.9, 
            "temperature": 0.7,
            "reasoningConfig": {
                "type": "enabled",
                "maxReasoningEffort": reasoning
            }
        } 
   
    request = {
        "messages": messages,
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=model_id, body=json.dumps(request))
    model_response = json.loads(response['body'].read().decode('utf-8'))
    if len(model_response["output"]["message"]["content"]) > 1:
        return model_response["output"]["message"]["content"][1]["text"]
    else:
        print(model_response["output"]["message"]["content"])
        return "Unexpected error occurred"

---

## Examples of skills

### Text Summarization

In this example, we download the full book Alice in Wonderland by Lewis Carroll, and ask the model to summarize it.

In [ ]:
## This cell may take 1-2 minute to run
import urllib.request
BOOK_LEN = 3200000

def download_url(url):
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read().decode("utf-8")
            return content
    except urllib.error.URLError as e:
        print(f"Error downloading URL: {e.reason}")
        return None

## download the book war and peace
book_content = download_url("https://www.gutenberg.org/cache/epub/2600/pg2600.txt")
print(f"The book contains roughly {len(book_content)} characters")
## Summarize the main events in the book. Nova Lite has a context window of 1MM tokens
pprint(invoke(f"Summarize the main events in following book in less than 500 words: {book_content[:BOOK_LEN]}", maxtokens=4096))

### Question Answering

This example show how the model can respond to questions about content of the book summarized in the previous cell. We will ask few questions that willbe answered from the book 'War and Peace' 

In [ ]:
## This cell may take a bit longer to run (1-2 Mins)
## Question 1 
print(
    invoke(f"""
**Question:**
Prince Bolkonsky’s pride keeps him from stooping to the politicking and self-promotion of other
staff officers and court officials—yet it also poisons his relationship with his wife, Lise, with his
sister, Marya, and, for much of the book, with his one-time fiancée, Natasha. How does a terrible
wound and a lingering illness change his attitude toward Marya, Natasha, and life in general?
DO NOT USE INFORMATION THAT IS NOT IN REFERENCE TEXTS!

**Reference Text:**
{book_content[:BOOK_LEN]}
""",
maxtokens=8092
    )
)


In [ ]:
## This cell may take a bit longer to run (1-2 Mins)
## Question 2
print(
    invoke(f"""
**Question:** 
What are the main events described in the book in Reference text below?
DO NOT USE INFORMATION THAT IS NOT IN REFERENCE TEXTS!

**Reference Text:**
{book_content[:BOOK_LEN]}
""",
reasoning="high"
    )
)


In [ ]:
## This cell may take a bit longer to run (1-2 Mins)
## Question 3
print(
    invoke(f"""
**Question:** 
Pierre’s passionate attachment to the world of ideas takes him from one enthusiasm to another—
from free-thinking Francophile to philanthropic Freemason to would-be assassin of the Antichrist,
Napoleon. What do you think Pierre is searching for? Why is he invariably ineffective when trying
to translate his enthusiasms into practical results? And how has he changed by the end of the
book
DO NOT USE INFORMATION THAT IS NOT IN REFERENCE TEXTS!

**Reference Text:**
{book_content[:BOOK_LEN]}
""",
reasoning="high"
    )
)

### Content Generation

This example shows how the model can generate creative text

In [ ]:
print(invoke(f"Write an ode to the commuting to work in less than 200 words"))

### Text Translation

Below we have a passage from Le Petit Prince by Antoine de Saint-Exupéry. We will ask model to identify the language, ask it (in english) to identify the book, ask it about the narrative.

In [ ]:
text = """J'ai sauté sur mes pieds comme si j'avais été frappé par la foudre. 
          J'ai bien frotté mes yeux. J'ai bien regardé. 
          Et j'ai vu un petit bonhomme tout à fait extraordinaire qui me considérait gravement. 
          Voilà le meilleur portrait que, plus tard, j'ai réussi à faire de lui. 
          Mais mon dessin, bien sûr, est beaucoup moins ravissant que le modèle. 
          Ce n'est pas ma faute. J'avais été découragé dans ma carrière de peintre par les grandes personnes, 
          à l'age de six ans, et je n'avais rien appris à dessiner, sauf les boas fermés et les boas ouverts. """
print(invoke(f"What language is this passage in: {text}"))
print(invoke(f"What book contains this passage: {text}"))
print(invoke(f"What did the character in this passage saw: {text}"))


### Text Analysis & Recommendations

In this example we show model's capability to understand text and sentiment

In [ ]:
review = """First off, the price alone probably led you here to start inquiring about reviews of quality. Fear not, 
            you get a lot more bag than the price suggests. Right out of the box, soft leather without much of that 
            leather smell. Medium brown color is classic and goes with plenty of styles. Overall, well made and provides 
            a lot of space for storage on your travels.
            I've noticed that the leather scent starts to increase with wear, which is quite pleasant. 
            The feel remains soft and durability seems to be quite good.

            Overall, great bag for the price considering you are getting a leather bag, 
            from a known solid brand name, all for under $100.00. The single only negative I have found with this bag is the latches 
            used to open and close the bag. They look nice and do their job, however they are a bit cheap and aren't 
            exactly quick and easy to use. They click into place just fine, but opening them is a fumbling challenge 
            as they do not have much give in the spring used to release the clip. I've gotten use to it, but don't 
            expect to quickly grab a business card at a moments notice as you'll be standing there till tomorrow 
            fumbling with the darn clip. A bit of an exaggeration that will change from user to user, but nonetheless 
            something worth mentioning.
            
            Don't over think this purchase. If you have the budget for a $500.00 bag, then by all means 
            keep searching as I do believe in getting what you pay for. However, the manufacture 
            retail price is $300, so you won't be scraping the bottom of the barrel. Grab this one while you can!"""
print(
    invoke(
        f"Answer with positive or negative, based on the sentiment on this review: {review}"
    )
)
print(invoke(f"List the things liked in this product review: {review}"))

### Dialogue and Conversation 

The following example shows how to use messages to create a back and forth chat.

In [ ]:
multi_turn_messages = []


def turn(prompt):
    multi_turn_messages.append({"role": "user", "content": [{"text": prompt}]})

    inference_config = {
        "maxTokens": 1000, 
        "topP": 0.9, 
        "temperature": 0.7,
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    } 

    request = {
        "messages": multi_turn_messages,
        "system": [{ "text": "Respond with short, concise answers" }],
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=DEFAULT_MODEL_ID, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    output_message = model_response["output"]["message"]
    multi_turn_messages.append(output_message)
    assistant_resp = output_message["content"][1]["text"]
    print(f"User: {prompt}")
    print(f"Assistant: {assistant_resp}")
    print("="*80)


pprint(turn("My name is Alice, I'm 30 years old, I moved to Seattle 9 years ago"))
pprint(turn("What's my name?"))
pprint(turn("Where do I live?"))
pprint(turn("At what age did I move to Seattle?"))

### Code Explanation & Generation

In the following example ask the model to generate code. 

In [ ]:
#Generate source code for a specified problem 
solution_response = invoke(
        """Write a 'odd_even' function in Python that takes in a list of numbers, and return a tuple with (odd numbers, even numbers).
        Do not explain your Approach or explanation in your response """,
        maxtokens=2048
    )
print(solution_response)

In [ ]:
# Nova Lite executes the source code to validate the code and results
response = invoke(f"""Run the code enclosed between ```python and ``` in a sandbox environment using code interpreter 
                      and show execution results
                      {solution_response}""",
                      maxtokens=4096)

print(response)

In [ ]:
## Functions to converse with Nova models with streaming option

def converse_with_nova_with_streaming(client, native_request, show_full_response=False):

    # Invoke the model and extract the response body.
    model_response = client.converse_stream(**native_request)
    # change the show_full_response flag to True to see full response
    if show_full_response:
        print("\n[Full Response]")
        print(json.dumps(model_response, indent=2))
    # Process the response stream
    stream = model_response.get("stream")
    if stream:
        for event in stream:
            if 'messageStart' in event:
                pass
            elif 'contentBlockDelta' in event:
                content_block_delta = event["contentBlockDelta"]
                if content_block_delta:
                    print(content_block_delta.get("delta").get("text"), end="")
    else:
        print("No response stream received.")

### Multi-turn classification tasks - Example 1

The following example shows how to use  multi-turns for classification.

In [ ]:
messages = [
  {"role": "user", "content": [{"text": """Your task is to classify the following texts into the appropriate categories: 
                                            Food, Entertainment, Health, Wealth, Other.""",}]},
  {"role": "user", "content": [{"text": "I love peperoni pizza."}]},
  {"role": "assistant", "content": [{"text": "Category: Food"}]},
  {"role": "user", "content": [{"text": "I enjoy watching movies."}]},
  {"role": "assistant", "content": [{"text": "Category: Entertainment"}]},
  {"role": "user", "content": [{"text": "I am going to the gym after this."}]},
  {"role": "assistant", "content": [{"text": "Category: Health"}]},
  {"role": "user", "content": [{"text": "I have $20 in my pocket."}]},
  {"role": "assistant", "content": [{"text": "Category: Wealth"}]},
  {"role": "user", "content": [{"text": "I may win a lottery today"}]}
]

inf_params = {
                "maxTokens": 10000, 
                "topP": 0.9, 
                "temperature": 0.7,
            } 

native_request = {
    "modelId": DEFAULT_MODEL_ID,
    "messages": messages,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova_with_streaming(client, native_request)


### Multi-turn classification tasks - Example 2

The following cell shows example of Not spam classification.

In [ ]:
## Examples are taken from opensource email-spam-classification database on huggingface 
## https://huggingface.co/datasets/UniqueData/email-spam-classification   

messages = [
  {"role": "user", "content": [{"text": """Your task is to classify the following email texts into the appropriate categories: 
                                            Spam, Not_Spam.

                                            You also explain your rationale briefly.""",}]},
  {"role": "user", "content": [{"text": """Hi James,  Have you claim your complimentary gift yet?  
                                           I've compiled in here a special astrology gift that predicts everything about you in the future?  
                                           This is your enabler to take the correct actions now.  
                                           >> Click here to claim your copy now >>  
                                           Claim yours now, and thank me later.   Love, Heather"""}]},
  {"role": "assistant", "content": [{"text": "Category: Spam"}]},

  {"role": "user", "content": [{"text": """Here's your GitHub launch code, @Mortyj420!  
                                           an octocat standing next to a rocket Continue signing up for GitHub by 
                                           entering the code below: 63024610  Open GitHub"""}]},
  {"role": "assistant", "content": [{"text": "Category: Not_Spam"}]},

  {"role": "user", "content": [{"text": """Hello Walid,  Do you have $7?  
                                           I ask because I'm about to do something STUPID.  
                                           The thing is, what I'm about to offer you isn't just amazing, 
                                           it's totally CRAZY.  Here's the deal:  Pay $7 and I'll give you $177 back.  
                                           Seriously, as if that wasn't good enough value, the products you'll 
                                           get have the potential to create you a $7000 a month income!  
                                           See exactly what you get for you $7 HERE.  Regards,  Charlie"""}]},
  {"role": "assistant", "content": [{"text": "Category: Spam"}]},

  {"role": "user", "content": [{"text": """ Model Casting Call Thank you for taking the time to register for the 
                                            Anambra Fashion Expo 2023 Model Call. We are thrilled to have received your information and 
                                            are excited to review your submission.  Our team will be carefully reviewing all 
                                            of the applications we receive over the next few weeks  Have you followed us on 
                                            our social media handles?  Remember, one of the prerequisites for qualification 
                                            is to follow all our social media accounts and share all our content using the 
                                            hashtag #AFE2023  You can follow us on Facebook, Instagram, and Twitter.  
                                            Facebook: https://facebook.com/AnambraFashionExpo2023 
                                            Instagram: https://instagram.com/anambrafashionexpo 
                                            Twitter: https://twitter.com/anambrafashion  In the meantime, we encourage you to 
                                            stay connected and keep an eye out for updates about the event. We will be posting! 
                                             Note: Create your personalized profile picture (DP) for the Model Casting of the 
                                             Anambra Fashion Expo 2023  You can create your DP using the following 
                                             link: https://getdp.co/afe2023  Best Regards,  Anambra Fashion Expo 2023 Team"""}]},
  {"role": "assistant", "content": [{"text": "Category: Not_Spam"}]},

  {"role": "user", "content": [{"text": """RE: Sales Executive  Dear Hiring Professional  I am contacting you to express my interest in 
                                           the Sales Executive position. After reviewing the position requirements, 
                                           I believe that my education and experience are a great match for this position.  
                                           I am an energetic and decisive negotiator with skills in goal-setting, sales and recommending 
                                           products. I have a natural talent for building immediate rapport with people and cultivating 
                                           productive connections. Likewise, I am fully capable of working with strong personalities and 
                                           navigating through high-pressure situations.  For more details, please review my attached resume. 
                                           I believe that I can be the Sales Executive you're looking for and welcome the opportunity to 
                                           speak with you at your earliest convenience.  Sincerely,  Zandi Tamane"""}]}
]

inf_params = {
                "maxTokens": 10000, "topP": 0.9, "temperature": 0.7,
            } 

native_request = {
    "modelId": DEFAULT_MODEL_ID,
    "messages": messages,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova_with_streaming(client, native_request)


### Multi-turn classification tasks - Example 3

The following cell shows example of spam classification

In [ ]:
## Examples are taken from opensource email-spam-classification database on huggingface 
## https://huggingface.co/datasets/UniqueData/email-spam-classification   

messages = [
  {"role": "user", "content": [{"text": """Your task is to classify the following email texts into the appropriate categories: 
                                            Spam, Not_Spam.

                                            You also explain your rationale briefly.""",}]},
  {"role": "user", "content": [{"text": """Hi James,  Have you claim your complimentary gift yet?  
                                           I've compiled in here a special astrology gift that predicts everything about you in the future?  
                                           This is your enabler to take the correct actions now.  
                                           >> Click here to claim your copy now >>  
                                           Claim yours now, and thank me later.   Love, Heather"""}]},
  {"role": "assistant", "content": [{"text": "Category: Spam"}]},

  {"role": "user", "content": [{"text": """Here's your GitHub launch code, @Mortyj420!  
                                           an octocat standing next to a rocket Continue signing up for GitHub by 
                                           entering the code below: 63024610  Open GitHub"""}]},
  {"role": "assistant", "content": [{"text": "Category: Not_Spam"}]},

  {"role": "user", "content": [{"text": """Hello Walid,  Do you have $7?  
                                           I ask because I'm about to do something STUPID.  
                                           The thing is, what I'm about to offer you isn't just amazing, 
                                           it's totally CRAZY.  Here's the deal:  Pay $7 and I'll give you $177 back.  
                                           Seriously, as if that wasn't good enough value, the products you'll 
                                           get have the potential to create you a $7000 a month income!  
                                           See exactly what you get for you $7 HERE.  Regards,  Charlie"""}]},
  {"role": "assistant", "content": [{"text": "Category: Spam"}]},

  {"role": "user", "content": [{"text": """ Model Casting Call Thank you for taking the time to register for the 
                                            Anambra Fashion Expo 2023 Model Call. We are thrilled to have received your information and 
                                            are excited to review your submission.  Our team will be carefully reviewing all 
                                            of the applications we receive over the next few weeks  Have you followed us on 
                                            our social media handles?  Remember, one of the prerequisites for qualification 
                                            is to follow all our social media accounts and share all our content using the 
                                            hashtag #AFE2023  You can follow us on Facebook, Instagram, and Twitter.  
                                            Facebook: https://facebook.com/AnambraFashionExpo2023 
                                            Instagram: https://instagram.com/anambrafashionexpo 
                                            Twitter: https://twitter.com/anambrafashion  In the meantime, we encourage you to 
                                            stay connected and keep an eye out for updates about the event. We will be posting! 
                                             Note: Create your personalized profile picture (DP) for the Model Casting of the 
                                             Anambra Fashion Expo 2023  You can create your DP using the following 
                                             link: https://getdp.co/afe2023  Best Regards,  Anambra Fashion Expo 2023 Team"""}]},
  {"role": "assistant", "content": [{"text": "Category: Not_Spam"}]},

  {"role": "user", "content": [{"text": """Hi Walid,  Do you listen to music on Spotify, YouTube, Amazon or Apple?  
                                           If you do - you qualify!  You could be making $50 for every song you stream...  
                                           All it takes is 3 steps...  Step 1: Create Your Account Create your account here  
                                           Step 2: Pick Your Favourite Artist Select from thousands of artists and vibe to the music  
                                           Step 3: Get Paid That's it, for every song you stream...  
                                           => Click here right now to start instantly  
                                           Regards,  Alex  --- ?? 
                                           Connect with us on Telegram: https://t.me/moneymakingcentral"""}]}
]

inf_params = {
                "maxTokens": 10000, "topP": 0.9, "temperature": 0.7,
            } 

native_request = {
    "modelId": DEFAULT_MODEL_ID,
    "messages": messages,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova_with_streaming(client, native_request)